## 1. Importing libraries

In [89]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
from pandas_profiling import ProfileReport
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

## 2. Reading data

In [90]:
df = pd.read_csv('Loan payments data.csv')
df.head()

,Loan_ID,loan_status,Principal,terms,effective_date,due_date,paid_off_time,past_due_days,age,education,Gender
0,xqd20166231,PAIDOFF,1000,30,9/8/2016,10/7/2016,9/14/2016 19:31,NaN,45,High School or Below,male
1,xqd20168902,PAIDOFF,1000,30,9/8/2016,10/7/2016,10/7/2016 9:00,NaN,50,Bechalor,female
2,xqd20160003,PAIDOFF,1000,30,9/8/2016,10/7/2016,9/25/2016 16:58,NaN,33,Bechalor,female
3,xqd20160004,PAIDOFF,1000,15,9/8/2016,9/22/2016,9/22/2016 20:00,NaN,27,college,male
4,xqd20160005,PAIDOFF,1000,30,9/9/2016,10/8/2016,9/23/2016 21:36,NaN,28,college,female


In [91]:
df1 = df

## 3. Data pre-processing and EDA

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
Loan_ID           500 non-null object
loan_status       500 non-null object
Principal         500 non-null int64
terms             500 non-null int64
effective_date    500 non-null object
due_date          500 non-null object
paid_off_time     400 non-null object
past_due_days     200 non-null float64
age               500 non-null int64
education         500 non-null object
Gender            500 non-null object
dtypes: float64(1), int64(3), object(7)
memory usage: 43.1+ KB


In [93]:
df.describe()

,Principal,terms,past_due_days,age
count,500.000000,500.000000,200.00000,500.000000
mean,943.200000,22.824000,36.01000,31.116000
std,115.240274,8.000064,29.38088,6.084784
min,300.000000,7.000000,1.00000,18.000000
25%,1000.000000,15.000000,3.00000,27.000000
50%,1000.000000,30.000000,37.00000,30.000000
75%,1000.000000,30.000000,60.00000,35.000000
max,1000.000000,30.000000,76.00000,51.000000


In [94]:
ProfileReport(df)

Number of variables,11
Number of observations,500
Total Missing (%),7.3%
Total size in memory,43.1 KiB
Average record size in memory,88.3 B
Numeric,4
Categorical,6
Boolean,0
Date,0
Text (Unique),1
Rejected,0


In [95]:
# Should handle imbalanced dataset
df['loan_status'].value_counts()

PAIDOFF               300
COLLECTION            100
COLLECTION_PAIDOFF    100
Name: loan_status, dtype: int64

In [96]:
le = LabelEncoder()
le.fit(df['loan_status'])
df1['loan_status'] = le.transform(df['loan_status'])

#le.inverse_transform([0, 0, 1, 2])

In [97]:
le.classes_

array(['COLLECTION', 'COLLECTION_PAIDOFF', 'PAIDOFF'], dtype=object)

In [98]:
df['Principal'].value_counts()

1000    377
800     111
300       6
500       3
900       2
700       1
Name: Principal, dtype: int64

In [99]:
principal = pd.get_dummies(df['Principal'], prefix='Principal')
principal

,Principal_300,Principal_500,Principal_700,Principal_800,Principal_900,Principal_1000
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,0,0,0,0,1
...,...,...,...,...,...,...
495,0,0,0,0,0,1
496,0,0,0,0,0,1
497,0,0,0,1,0,0
498,0,0,0,0,0,1


In [100]:
bias = ['Principal_300', 'Principal_500', 'Principal_700', 'Principal_900']
principal['Principal_bias'] = principal[bias].sum(axis=1)
principal.drop(bias,1,inplace=True)
principal.head()

,Principal_800,Principal_1000,Principal_bias
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0


In [101]:
df1.drop(['Principal'],1,inplace=True)
df1 = pd.concat([df1,principal],1)
df1.head()

,Loan_ID,loan_status,terms,effective_date,due_date,paid_off_time,past_due_days,age,education,Gender,Principal_800,Principal_1000,Principal_bias
0,xqd20166231,2,30,9/8/2016,10/7/2016,9/14/2016 19:31,NaN,45,High School or Below,male,0,1,0
1,xqd20168902,2,30,9/8/2016,10/7/2016,10/7/2016 9:00,NaN,50,Bechalor,female,0,1,0
2,xqd20160003,2,30,9/8/2016,10/7/2016,9/25/2016 16:58,NaN,33,Bechalor,female,0,1,0
3,xqd20160004,2,15,9/8/2016,9/22/2016,9/22/2016 20:00,NaN,27,college,male,0,1,0
4,xqd20160005,2,30,9/9/2016,10/8/2016,9/23/2016 21:36,NaN,28,college,female,0,1,0


In [102]:
df['terms'].value_counts()

30    272
15    207
7      21
Name: terms, dtype: int64

In [103]:
terms = pd.get_dummies(df['terms'],drop_first=True, prefix='terms')
df1.drop(['terms'],1,inplace=True)
df1 = pd.concat([df1,terms],1)
df1.head()

,Loan_ID,loan_status,effective_date,due_date,paid_off_time,past_due_days,age,education,Gender,Principal_800,Principal_1000,Principal_bias,terms_15,terms_30
0,xqd20166231,2,9/8/2016,10/7/2016,9/14/2016 19:31,NaN,45,High School or Below,male,0,1,0,0,1
1,xqd20168902,2,9/8/2016,10/7/2016,10/7/2016 9:00,NaN,50,Bechalor,female,0,1,0,0,1
2,xqd20160003,2,9/8/2016,10/7/2016,9/25/2016 16:58,NaN,33,Bechalor,female,0,1,0,0,1
3,xqd20160004,2,9/8/2016,9/22/2016,9/22/2016 20:00,NaN,27,college,male,0,1,0,1,0
4,xqd20160005,2,9/9/2016,10/8/2016,9/23/2016 21:36,NaN,28,college,female,0,1,0,0,1


In [104]:
df1.drop(['Loan_ID','effective_date', 'due_date', 'paid_off_time', 'past_due_days'],1,inplace=True)

In [105]:
mmc = MinMaxScaler()
df1['age'] = mmc.fit_transform(df['age'].values.reshape(-1, 1))
df1.head()

,loan_status,age,education,Gender,Principal_800,Principal_1000,Principal_bias,terms_15,terms_30
0,2,0.818182,High School or Below,male,0,1,0,0,1
1,2,0.969697,Bechalor,female,0,1,0,0,1
2,2,0.454545,Bechalor,female,0,1,0,0,1
3,2,0.272727,college,male,0,1,0,1,0
4,2,0.303030,college,female,0,1,0,0,1


In [106]:
education = pd.get_dummies(df['education'],drop_first=True, prefix='education')
df1.drop(['education'],1,inplace=True)
df1 = pd.concat([df1,education],1)
df1.head()

,loan_status,age,Gender,Principal_800,Principal_1000,Principal_bias,terms_15,terms_30,education_High School or Below,education_Master or Above,education_college
0,2,0.818182,male,0,1,0,0,1,1,0,0
1,2,0.969697,female,0,1,0,0,1,0,0,0
2,2,0.454545,female,0,1,0,0,1,0,0,0
3,2,0.272727,male,0,1,0,1,0,0,0,1
4,2,0.303030,female,0,1,0,0,1,0,0,1


In [107]:
# Handle unbalanced data
df['Gender'].value_counts()

male      423
female     77
Name: Gender, dtype: int64

In [108]:
le_Gender = LabelEncoder()
le_Gender.fit(df['Gender'])
df1['Gender'] = le_Gender.transform(df['Gender'])

#le_Gender.inverse_transform([0, 0, 1, 2])

In [109]:
df1.head()

,loan_status,age,Gender,Principal_800,Principal_1000,Principal_bias,terms_15,terms_30,education_High School or Below,education_Master or Above,education_college
0,2,0.818182,1,0,1,0,0,1,1,0,0
1,2,0.969697,0,0,1,0,0,1,0,0,0
2,2,0.454545,0,0,1,0,0,1,0,0,0
3,2,0.272727,1,0,1,0,1,0,0,0,1
4,2,0.303030,0,0,1,0,0,1,0,0,1


In [110]:
X = df1.drop(['loan_status'],1)
y = df1['loan_status']

In [111]:
smote = SMOTE()
X_sm, y_sm = smote.fit_sample(X, y)

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## 4. XGBoost model

### Model building before SMOTE

In [113]:
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

print('Test Confusion matrix:',confusion_matrix(y_test, y_pred))

Test Confusion matrix: [[ 2  6 22]
 [ 1  5 21]
 [14  9 45]]


In [114]:
y_avg = np.array([2] * len(y_test))
print('Null Confusion matrix:',confusion_matrix(y_test, y_avg))

Null Confusion matrix: [[ 0  0 30]
 [ 0  0 27]
 [ 0  0 68]]


In [115]:
print('Test accuracy:',accuracy_score(y_test, y_pred))
print('Null accuracy:',accuracy_score(y_test, y_avg))

Test accuracy: 0.416
Null accuracy: 0.544


In [116]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

[[ 2  6 22]
 [ 1  5 21]
 [14  9 45]]
              precision    recall  f1-score   support

           0       0.12      0.07      0.09        30
           1       0.25      0.19      0.21        27
           2       0.51      0.66      0.58        68

    accuracy                           0.42       125
   macro avg       0.29      0.30      0.29       125
weighted avg       0.36      0.42      0.38       125



In [117]:
final_df = pd.DataFrame()
final_df["Prediction"] = y_pred
final_df["Actual"] = y_test.values
final_df

,Prediction,Actual
0,2,0
1,0,2
2,2,0
3,2,2
4,2,2
...,...,...
120,2,2
121,1,2
122,1,0
123,0,2


### Model building after SMOTE

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.25, random_state=42)

xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

#print('Null accuracy:',accuracy_score(y_test, y_avg))
print('Test accuracy:',accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

Test accuracy: 0.49777777777777776
[[31 18 29]
 [18 36 20]
 [13 15 45]]
              precision    recall  f1-score   support

           0       0.50      0.40      0.44        78
           1       0.52      0.49      0.50        74
           2       0.48      0.62      0.54        73

    accuracy                           0.50       225
   macro avg       0.50      0.50      0.50       225
weighted avg       0.50      0.50      0.49       225



In [119]:
y_avg = np.array([0] * len(y_test))
print('Null accuracy:',accuracy_score(y_test, y_avg))

Null accuracy: 0.3466666666666667


In [120]:
#y_test = pd.DataFrame(y_test)
y_test.value_counts()

0    78
1    74
2    73
Name: loan_status, dtype: int64

In [121]:
final_df1 = pd.DataFrame()
final_df1["Prediction"] = y_pred
final_df1["Actual"] = y_test.values
final_df1

,Prediction,Actual
0,1,2
1,0,1
2,2,2
3,0,0
4,0,2
...,...,...
220,1,2
221,2,0
222,2,1
223,1,2
